In [23]:
import os
import pandas as pd
import numpy as np

#Load expression matrices and join
# load all table files
path_to_files = '/home/mar/Documents/TFM/GitHub/HB_PublicData/1-Obtaining-DEGs-for-HB/DEGs_HB/Study_Variability/output/'
lst_expmat = []
for filen in [x for x in os.listdir(path_to_files) if '.csv' in x]:
    lst_expmat.append(pd.read_csv(path_to_files+filen, delimiter= ",").set_index(['genes']))

data = pd.concat(lst_expmat, axis=1, join='inner')
data = data.reset_index()

,genes,log2FoldChange,NT,T,T-NT,log2FoldChange,NT,T,T-NT,log2FoldChange,...,T,T-NT,log2FoldChange,NT,T,T-NT,log2FoldChange,NT,T,T-NT
0,TRH,7.983702,9.105488,2314.907619,2305.802131,0.635647,52.611964,81.775390,29.163426,5.088671,...,1133.981030,1100.792011,6.342405,17.989449,1454.454864,1436.465415,-4.248107,172.278049,2.521102,-169.756947
1,SHISA6,7.250302,0.865932,133.962856,133.096924,-1.782046,18.604204,5.470239,-13.133965,7.040968,...,220.638632,219.013193,5.472822,2.345664,102.339883,99.994220,-4.032376,143.498439,1.681667,-141.816772
2,CST1,6.904941,5.415136,654.583693,649.168558,0.349335,21.645162,312.087706,290.442543,6.681713,...,923.755017,914.816021,5.679612,28.953963,1480.813326,1451.859364,-4.413528,544.529307,25.488466,-519.040841
3,MATN3,6.870514,16.357554,1926.395154,1910.037600,0.802203,170.475404,297.310326,126.834922,5.490260,...,1559.236609,1524.654181,5.054663,19.565280,647.967951,628.402671,-2.385505,406.661303,77.724160,-328.937143
4,MYH4,-6.341043,251.964043,112.440023,-139.524020,-0.484586,0.375447,0.187161,-0.188287,-4.157224,...,9.340663,-155.835564,2.800920,5.473215,248.329239,242.856024,2.199080,33.390858,153.520002,120.129144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15918,RSPRY1,-0.000166,913.699807,914.058531,0.358724,1.752574,34.208363,116.216903,82.008540,-0.150722,...,822.448424,-89.941242,0.212721,235.706295,272.965873,37.259578,0.043851,1334.673964,1375.724734,41.050770
15919,DDX6,0.000139,2183.983237,2183.879347,-0.103890,-2.816572,2681.949895,380.790594,-2301.159301,0.286464,...,2813.520915,506.862588,0.201808,1066.674447,1225.575102,158.900654,-0.176873,6471.007381,5723.684177,-747.323204
15920,ZC3H4,0.000077,945.151486,945.583951,0.432465,-0.440759,135.240514,99.754986,-35.485528,0.005888,...,781.614255,2.792921,0.432659,291.525146,394.108650,102.583504,0.009027,574.373888,578.233317,3.859430
15921,SLC35G5,-0.000032,6.320113,6.214360,-0.105753,1.300273,0.157994,1.122200,0.964206,0.744918,...,2.668759,0.983358,-0.502097,21.129056,15.029248,-6.099808,-0.000825,8.006613,8.529515,0.522902


## LFC

In [25]:
lfc = data[['log2FoldChange']]
lfc.columns = ['A','B','C','D','E']
lfc_melt = pd.melt(lfc.reset_index(), id_vars=['index'], value_vars=['A', 'B', 'C', 'D','E'])
# replace column names
lfc_melt.columns = ['index', 'studies', 'value']

,index,studies,value
0,0,A,7.983702
1,1,A,7.250302
2,2,A,6.904941
3,3,A,6.870514
4,4,A,-6.341043
...,...,...,...
79610,15918,E,0.043851
79611,15919,E,-0.176873
79612,15920,E,0.009027
79613,15921,E,-0.000825


In [ ]:
# generate a boxplot to see the data distribution by treatments. Using boxplot, we can 
# easily detect the differences between different treatments
import matplotlib.pyplot as plt
import seaborn as sns
ax = sns.boxplot(x='studies', y='value', data=lfc_melt, color='#99c2a2')
ax = sns.swarmplot(x="studies", y="value", data=lfc_melt, color='#7d0013')
plt.show()

In [14]:
import scipy.stats as stats
# stats f_oneway functions takes the groups as input and returns ANOVA F and p value
fvalue, pvalue = stats.f_oneway(lfc['A'], lfc['B'], lfc['C'], lfc['D'], lfc['E'])
print(fvalue, pvalue)
# ANOVA table using bioinfokit v1.0.3 or later (it uses wrapper script for anova_lm)
from bioinfokit.analys import stat
res = stat()
res.anova_stat(df=lfc_melt, res_var='value', anova_model='value ~ C(studies)')
res.anova_summary

112.40923434088532 9.740875350275957e-96


,df,sum_sq,mean_sq,F,PR(>F)
C(studies),4.0,705.728714,176.432179,112.409234,9.740875e-96
Residual,79610.0,124952.063033,1.569552,NaN,NaN


In [37]:
## T

In [28]:
T = data[['T']]
T.columns = ['A','B','C','D','E']
T_melt = pd.melt(T.reset_index(), id_vars=['index'], value_vars=['A', 'B', 'C', 'D','E'])
# replace column names
T_melt.columns = ['index', 'studies', 'value']

,index,studies,value
0,0,A,2314.907619
1,1,A,133.962856
2,2,A,654.583693
3,3,A,1926.395154
4,4,A,112.440023
...,...,...,...
79610,15918,E,1375.724734
79611,15919,E,5723.684177
79612,15920,E,578.233317
79613,15921,E,8.529515


In [ ]:
# generate a boxplot to see the data distribution by treatments. Using boxplot, we can 
# easily detect the differences between different treatments
import matplotlib.pyplot as plt
import seaborn as sns
ax = sns.boxplot(x='studies', y='value', data=T_melt, color='#99c2a2')
ax = sns.swarmplot(x="studies", y="value", data=T_melt, color='#7d0013')
plt.show()

In [16]:
import scipy.stats as stats
# stats f_oneway functions takes the groups as input and returns ANOVA F and p value
fvalue, pvalue = stats.f_oneway(T['A'], T['B'], T['C'], T['D'], T['E'])
print(fvalue, pvalue)
# ANOVA table using bioinfokit v1.0.3 or later (it uses wrapper script for anova_lm)
from bioinfokit.analys import stat
res = stat()
res.anova_stat(df=T_melt, res_var='value', anova_model='value ~ C(studies)')
res.anova_summary

29.205806113746284 2.6546001498375815e-24


,df,sum_sq,mean_sq,F,PR(>F)
C(studies),4.0,3.943213e+10,9.858033e+09,29.205806,2.654600e-24
Residual,79610.0,2.687130e+13,3.375367e+08,NaN,NaN


## NT

In [32]:
NT = data[['NT']]
NT.columns = ['A','B','C','D','E']
NT_melt = pd.melt(NT.reset_index(), id_vars=['index'], value_vars=['A', 'B', 'C', 'D','E'])
# replace column names
NT_melt.columns = ['index', 'studies', 'value']

In [ ]:
# generate a boxplot to see the data distribution by treatments. Using boxplot, we can 
# easily detect the differences between different treatments
import matplotlib.pyplot as plt
import seaborn as sns
ax = sns.boxplot(x='studies', y='value', data=NT_melt, color='#99c2a2')
ax = sns.swarmplot(x="studies", y="value", data=NT_melt, color='#7d0013')
plt.show()

In [ ]:
import scipy.stats as stats
# stats f_oneway functions takes the groups as input and returns ANOVA F and p value
fvalue, pvalue = stats.f_oneway(NT['A'], NT['B'], NT['C'], NT['D'], NT['E'])
print(fvalue, pvalue)
# ANOVA table using bioinfokit v1.0.3 or later (it uses wrapper script for anova_lm)
from bioinfokit.analys import stat
res = stat()
res.anova_stat(df=NT_melt, res_var='value', anova_model='value ~ C(studies)')
res.anova_summary

## T - NT

In [30]:
T_NT = data[['T-NT']]
T_NT.columns = ['A','B','C','D','E']
T_NT_melt = pd.melt(T_NT.reset_index(), id_vars=['index'], value_vars=['A','B','C','D','E'])
# replace column names
T_NT_melt.columns = ['index', 'studies', 'value']

In [ ]:
# generate a boxplot to see the data distribution by treatments. Using boxplot, we can 
# easily detect the differences between different treatments
import matplotlib.pyplot as plt
import seaborn as sns
ax = sns.boxplot(x='studies', y='value', data=T_NT_melt, color='#99c2a2')
ax = sns.swarmplot(x="studies", y="value", data=T_NT_melt, color='#7d0013')
plt.show()

In [31]:
import scipy.stats as stats
# stats f_oneway functions takes the groups as input and returns ANOVA F and p value
fvalue, pvalue = stats.f_oneway(T_NT['A'], T_NT['B'], T_NT['C'], T_NT['D'], T_NT['E'])
print(fvalue, pvalue)
# ANOVA table using bioinfokit v1.0.3 or later (it uses wrapper script for anova_lm)
from bioinfokit.analys import stat
res = stat()
res.anova_stat(df=T_NT_melt, res_var='value', anova_model='value ~ C(studies)')
res.anova_summary

21.082589443362053 2.1491760137105988e-17


,df,sum_sq,mean_sq,F,PR(>F)
C(studies),4.0,2.522465e+10,6.306163e+09,21.082589,2.149176e-17
Residual,79610.0,2.381271e+13,2.991171e+08,NaN,NaN
